In [1]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
import pandas as pd
import sqlite3
import os

## init load of data into sqlite

In [ ]:
os.getcwd()

In [ ]:
# change to where csvs are located
df_mthstatus = pd.read_csv('../../../../ss/ssData/tkcmonthlystatus.csv')
df_manpower = pd.read_csv('../../../../ss/ssData/tkcgmp.csv')

In [ ]:
[df_mthstatus.shape, df_manpower.shape]

In [ ]:
def remove_last_space(x):
    return x[:-1] if x[-1] == ' ' else x

def remove_last_dash(x):
    return x[:-1] if x[-1] == '-' else x 

In [ ]:
# this so we can join on contract
df_manpower['contract'] = df_manpower['project_number'].apply(remove_last_space)
df_mthstatus['contract'] = df_mthstatus['contract'].apply(remove_last_dash)

In [ ]:
# write to sqlite db
df_mthstatus.to_sql("mthstatus", con, if_exists="replace")
df_manpower.to_sql("manpower", con, if_exists="replace")

In [ ]:
# change to where tkc.db is located
sqldb_str = "/Users/scottschmidt/sqlitedbs/tkc.db"

In [ ]:
con = sqlite3.connect(sqldb_str)

## test the db

In [ ]:
# change to where tkc.db is located
sqldb_str = "/Users/scottschmidt/sqlitedbs/tkc.db"

In [ ]:
con = sqlite3.connect(sqldb_str)

In [ ]:
cursor = con.cursor()

In [ ]:
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

In [ ]:
cursor.execute(sql_query)
print(cursor.fetchall())

In [ ]:
cursor.execute("Select * from manpower limit 10")
print(cursor.fetchall())

In [ ]:
cursor.execute("Select * from mthstatus limit 10")
print(cursor.fetchall())

In [ ]:
for row in con.execute("pragma table_info('manpower')").fetchall():
    print(row)

In [ ]:
con.close()

## now test langchain

In [2]:

sqldb_str = "sqlite:////Users/scottschmidt/sqlitedbs/tkc.db"
db = SQLDatabase.from_uri(sqldb_str)
llm = OpenAI(temperature=0)

In [3]:
db.get_usable_table_names()

{'manpower', 'mthstatus'}

In [4]:
# db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [5]:
sa = db_chain.run("How many unique contracts are there in table mthstatus?")



> Entering new SQLDatabaseChain chain...
How many unique contracts are there in table mthstatus?
SQLQuery: SELECT COUNT(DISTINCT contract) FROM mthstatus;
SQLResult: [(1051,)]
Answer: There are 1051 unique contracts in table mthstatus.
> Finished chain.


In [ ]:
db_chain